In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import ruamel.yaml as yaml
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
import subprocess
import datetime
import sys
import numpy as np
import logging

sys.path.append("../studiogan")

from maxent_gan.utils.train.trainer import Trainer
from maxent_gan.utils.train.loss import LossRegistry
from maxent_gan.utils.general_utils import DotConfig, random_seed, seed_worker, ROOT_DIR
from maxent_gan.feature.utils import create_feature
from maxent_gan.models.utils import GANWrapper
from maxent_gan.sample import MaxEntSampler
from maxent_gan.distribution import DistributionRegistry
from maxent_gan.utils.callbacks import CallbackRegistry
from maxent_gan.datasets.utils import get_dataset

2022-04-18 15:33:10.582904: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# import sys
# sys.path.append('..')

In [3]:
from easydict import EasyDict as edict

args = edict()

configs = ['configs/exp_configs/mlp-cmd.yml', 'configs/targets/prior.yml', 'configs/gan_configs/ring-mlp.yml', 'configs/feature_configs/cmd_2d.yml', 'configs/train_2d_iw.yml']
configs = [Path(ROOT_DIR, x).as_posix() for x in configs]
args.configs = configs

In [4]:
params = yaml.round_trip_load(Path(args.configs[0]).open("r"))

In [5]:
proc = subprocess.Popen("/bin/bash", stdin=subprocess.PIPE, stdout=subprocess.PIPE)
out, err = proc.communicate(
    (
        " ".join(
            [
                "echo",
                '"' + str(yaml.round_trip_dump(params)) + '"',
                "|",
                "cat - ",
                *args.configs[1:],
            ]
        )
    ).encode("utf-8")
)
config = yaml.round_trip_load(out.decode("utf-8"))
config = DotConfig(config)

In [6]:
gan = GANWrapper(config.gan_config, device=config.device, load_weights=False)

In [7]:
dataset_stuff = get_dataset(
        config.gan_config.dataset.name,
        mean=config.gan_config.train_transform.Normalize.mean,
        std=config.gan_config.train_transform.Normalize.std,
        seed=config.seed,
        **config.gan_config.dataset.params,
    )
dataset = dataset_stuff["dataset"]

dataloader = DataLoader(
    dataset, 
    batch_size=config.train_batch_size, 
    shuffle=True, 
    num_workers=4, 
    worker_init_fn=seed_worker)

In [8]:
device = config.device

In [9]:
config.feature.params['ref_stats_path'] = Path(ROOT_DIR, config.feature.params.ref_stats_path)

In [10]:
config.feature.params.ref_stats_path

PosixPath('/home/lagutin/constrained_gen/maxent_gan/stats/CMDFeature_2d.npz')

In [20]:
ref_dist = DistributionRegistry.create(config.sample_params.distribution.name, gan=gan)
# if config.sample_params.distribution.name == 'PriorTarget' and config.feature.name == 'DumbFeature':
#     sampler = None
# else:
feature_dataloader = DataLoader(dataset, batch_size=config.data_batch_size)
feature = create_feature(config, gan, feature_dataloader, dataset_stuff, None, device)
sampler = MaxEntSampler(gen=gan.gen, ref_dist=ref_dist, feature=feature, **config.sample_params.params, sweet_init=True)

100%|██████████| 13/13 [00:00<00:00, 249.08it/s]

defaultdict(<function evaluate.<locals>.<lambda> at 0x7fd2a40d4c10>, {0: array([-8.9779633e-06, -1.5134773e-04,  1.1799289e-01,  1.1753958e-01,
       -9.5862197e-06,  1.7874552e-05], dtype=float32)})


In [21]:
feature.weight

[tensor([-1.9473e+01, -2.4499e+00, -6.9691e+01, -6.9427e+01,  6.8881e-03,
         -1.0582e-02], device='cuda:0')]